In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pathlib

# Kiểm tra và thiết lập device (GPU nếu có, ngược lại dùng CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Tải dữ liệu
training_dir = pathlib.Path('radar-signal-classification/training_set')

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

full_dataset = datasets.ImageFolder(root=str(training_dir), transform=transform)

# Chia dữ liệu thành tập train và validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class_names = full_dataset.classes  
num_classes = len(class_names)

# Tạo DataLoader với pin_memory=True để tối ưu hóa truyền dữ liệu lên GPU
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True)

print(f"Classes: {class_names}")
print(f"Training samples: {train_size}, Validation samples: {val_size}")

# Định nghĩa mô hình CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)  
        
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))  
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))  
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))  
        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Khởi tạo mô hình và chuyển lên GPU
model = SimpleCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss().to(device)  # Chuyển loss function lên GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Hàm train với các thao tác trên GPU
def train(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    train_loss, val_loss, train_acc, val_acc = [], [], [], []
    
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            # Chuyển dữ liệu lên GPU
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        train_loss.append(running_loss / len(train_loader))
        train_acc.append(correct / total)

        # Validation phase
        model.eval()
        val_running_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss.append(val_running_loss / len(val_loader))
        val_acc.append(correct / total)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f} - "
              f"Val Loss: {val_loss[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return model

# Huấn luyện mô hình
epochs = 5
model = train(model, train_loader, val_loader, criterion, optimizer, epochs)

# Lưu mô hình
example_input = torch.randn(1, 3, 128, 128).to(device)  
traced_model = torch.jit.trace(model, example_input)
traced_model.save("22119129.pt")
print("Model saved as 22119129.pt")

# Giải phóng bộ nhớ GPU (nếu cần)
torch.cuda.empty_cache()
        

Using device: cuda
Classes: ['B-FM', 'Barker', 'CPFSK', 'DSB-AM', 'GFSK', 'LFM', 'Rect', 'SSB-AM']
Training samples: 5120, Validation samples: 1280
Epoch 1/5 - Train Loss: 1.3213, Train Acc: 0.5906 - Val Loss: 0.5078, Val Acc: 0.7844
Epoch 2/5 - Train Loss: 0.6201, Train Acc: 0.7172 - Val Loss: 0.3955, Val Acc: 0.8219
Epoch 3/5 - Train Loss: 0.4787, Train Acc: 0.7686 - Val Loss: 0.3713, Val Acc: 0.8289
Epoch 4/5 - Train Loss: 0.4159, Train Acc: 0.7979 - Val Loss: 0.4511, Val Acc: 0.7703
Epoch 5/5 - Train Loss: 0.4179, Train Acc: 0.7965 - Val Loss: 0.3411, Val Acc: 0.8406
Model saved as 22119129.pt
